In [1]:
!pip install ultralytics mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


In [1]:
!nohup mlflow server --backend-store-uri postgresql:// --host 0.0.0.0 &

nohup: appending output to 'nohup.out'


In [8]:
!yolo settings mlflow=False

💡 Learn about settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings
Printing '/root/.config/Ultralytics/settings.yaml'

settings_version: 0.0.4
datasets_dir: /content/datasets
weights_dir: weights
runs_dir: runs
uuid: 569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb
sync: true
api_key: ''
clearml: true
comet: true
dvc: true
hub: true
mlflow: false
neptune: true
raytune: true
tensorboard: true
wandb: true


In [2]:
import ultralytics
from ultralytics import YOLO
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics.nn.tasks import DetectionModel

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import mlflow

In [4]:
mlflow.set_tracking_uri("http://0.0.0.0:5000")
mlflow.start_run(run_name="ultralytics yolo distil params")

<ActiveRun: >

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
class MyCustomModelDistillation(DetectionModel):
    def __init__(self, teacher_model: str, device: torch.device, T=1, **kwargs):
        super().__init__(**kwargs)

        self.teacher_model = DetectionModel(teacher_model).to(device)
        self.distil_criterion = nn.KLDivLoss(reduction="batchmean")
        self.mae_loss = nn.L1Loss()
        self.T = 1

    def loss(self, batch, preds=None):
        if not hasattr(self, 'criterion'):
            self.criterion = self.init_criterion()

        preds = self.forward(batch['img']) if preds is None else preds
        # print(preds)
        criterion = self.criterion(preds, batch)
        distil_part = 0.
        if len(preds) == 3:
            with torch.no_grad():
                teacher_output = self.teacher_model.forward(batch['img'])
            distil_part = 0
            # for i in range(3):
            #     distil_part += self.criterion(F.softmax(preds[i].detach() / self.T, dim=-1), F.softmax(teacher_output[i].detach() / self.T, dim=-1))
            # distil_part = self.criterion(F.softmax(preds, dim=-1), F.softmax(teacher_output, dim=-1))
            # distil_part = self.T**2 * distil_part
            for i in range(3):
                distil_part += self.mae_loss(preds[i], teacher_output[i].detach())
        new_criterion = (criterion[0] + distil_part, criterion[1])
        mlflow.log_metric("loss", new_criterion[0])
        return new_criterion


class CustomTrainerDistillation(DetectionTrainer):
    def get_model(self, cfg=None, weights=None, verbose=True):
        """Return a YOLO detection model."""
        model = MyCustomModelDistillation("yolov8l.yaml", device, cfg=cfg, nc=self.data['nc'], verbose=False)
        if weights:
            model.load(weights)
        return model

In [9]:
trainer = CustomTrainerDistillation(overrides={'data': 'coco128.yaml', 'model': 'yolov8n.yaml', 'epochs': 100})
trainer.train()
trained_model = trainer.best  # get best model

Ultralytics YOLOv8.0.199 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=coco128.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False

MlflowException: ignored

In [ ]:
model_distilled = YOLO('runs/detect/train8/weights/best.pt')

In [ ]:
result_distilled = model_distilled.val(data='coco128.yaml')

In [ ]:
for key, val in result_distilled.results_dict.items():
    new_key = key.replace("(", "").replace(")", "")
    mlflow.log_metric(new_key, val)
for key, val in result_distilled.speed.items():
    new_key = key.replace("(", "").replace(")", "")
    mlflow.log_metric(new_key, val)

In [ ]:
mlflow.end_run()